In [ ]:
import osmnx as ox
import networkx as nx
import ast
import sys
# Change this for your local machine
sys.path.append("C:\\Users\\gabri\\Documents\\CMOR492-DWS")
import json

In [4]:
G = ox.load_graphml("road_net_2.graphml")

In [ ]:
with open("solutions\\y_sol_period_1.json", "r") as f:
    y_sol_1 = {ast.literal_eval(key): value for key, value in json.load(f).items()}

with open("solutions\\z_sol_period_1.json", "r") as f:
    z_sol_1 = {ast.literal_eval(key): value for key, value in json.load(f).items()}

In [ ]:
possible_treatment_nodes = 

In [ ]:
node_colors = []
for node in G.nodes:
    if y_sol_1[node] == 1:
        node_colors.append("yellow")
    else if node in y_sol_1.keys(): #i.e. 
        node_colors.append("#666666")
    else:
        node_colors.append("#336699")


edge_colors = ['white' if z_0[e[0], e[1]] == 1 else 'purple' for e in G.edges]

# node_colors = ['r' if node in treatment_nodes else '#336699' for node in G.nodes()]

node_sizes = [50 if y_sol[node] == 1 else 15 for node in G.nodes()]

ox.plot_graph(G, node_color=node_colors, node_size=node_sizes, edge_color=edge_colors, edge_linewidth=0.5, bgcolor="#000000")

In [3]:
y_sol

{59128065: 1.0,
 59133314: 1.0,
 59116291: 0.0,
 59121412: 0.0,
 59081605: 0.0,
 59126150: 1.0,
 59133192: 1.0,
 59117577: 0.0,
 59091084: 1.0,
 59123853: 1.0,
 59110286: 1.0,
 59116300: 0.0,
 59129101: 0.0,
 59146899: 1.0,
 59095700: 0.0,
 59124248: 0.0,
 59096602: 0.0,
 59110431: 0.0,
 59079200: 1.0,
 3584201894: 0.0,
 4231830135: 1.0,
 59123881: 1.0,
 59101100: 0.0,
 59079983: 0.0,
 59090484: 1.0,
 59146293: 1.0,
 59129525: 1.0,
 59118652: 1.0,
 59103172: 0.0,
 59101385: 1.0,
 59146313: 0.0,
 59110347: 0.0,
 59158864: 1.0,
 59133777: 1.0,
 59146231: 1.0,
 59080788: 0.0,
 59081566: 1.0,
 9941438054: 1.0,
 59147494: 1.0,
 59124459: 1.0,
 59115372: 1.0,
 59153773: 1.0,
 4230838774: 1.0,
 59164150: 0.0,
 59113465: 0.0,
 59107322: 1.0,
 59097341: 1.0}